## Import torch and model

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = '1'

In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

import numpy as np

In [3]:
import sys
sys.path.append("../common")

from model_generator import ModelGenerator
from net import Net

## Set hyper params

In [4]:
batch_size = 128
num_epoch  = 180

lr = 0.1
gamma = 0.2             # learning rate decay
weight_decay = 0.0001

## for SGD
opt_momentum = 0.9
opt_nesterov = True

dropout_on = False
batchnorm_on = True 

scheduler_step_size = [60, 120, 150]

pretrained_model       = './cifar100_vgg16_pretrained.pth'

## Load dataset

In [5]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.2023, 0.1994, 0.2010))])

transform_train = transforms.Compose(
    [transforms.RandomHorizontalFlip(),
     transforms.RandomCrop(32, 4),
     transforms.ToTensor(),
     transforms.Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.2023, 0.1994, 0.2010))])

trainset = torchvision.datasets.CIFAR100(root='./data', train=True,
                                        download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR100(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


## VGG16 model

In [6]:
model_gen = ModelGenerator(dropout = dropout_on, batchnorm = batchnorm_on)

model_gen.CifarVgg16Config(cifar = 100)
model = model_gen.GetCifarVgg16()

net = Net(model)

net.Gpu()

In [7]:
criterion = nn.CrossEntropyLoss()

optimizer = optim.SGD(net.GetTotalParams(), lr=lr, weight_decay=weight_decay, momentum=opt_momentum, nesterov=opt_nesterov )

scheduler = lr_scheduler.MultiStepLR(optimizer, milestones = scheduler_step_size, gamma = gamma)

In [8]:
for epoch in range(num_epoch):  # loop over the dataset multiple times

    running_loss = 0.0
    scheduler.step()
    net.TrainMode()
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data

        # wrap them in Variable
        inputs = Variable(inputs.cuda())
        labels = Variable(labels.cuda())

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        #scheduler.step()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss = (running_loss * i + loss.cpu().data.numpy()) / (i+1)

    correct = 0
    total = 0
    net.TestMode()
    for data in testloader:
        images, labels = data
        outputs = net(Variable(images.cuda()))
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels.cuda()).sum()
        
    print('%d epoch end, loss: %3.6f, Test Acc: %4.2f %%' %(epoch + 1, running_loss, 100 * correct / total))
    
print('\nTraining is finished!')

1 epoch end, loss: 4.322362, Test Acc: 6.41 %
2 epoch end, loss: 3.851555, Test Acc: 10.36 %
3 epoch end, loss: 3.574304, Test Acc: 13.85 %
4 epoch end, loss: 3.353385, Test Acc: 17.74 %
5 epoch end, loss: 3.120760, Test Acc: 19.49 %
6 epoch end, loss: 2.926459, Test Acc: 23.38 %
7 epoch end, loss: 2.727441, Test Acc: 24.28 %
8 epoch end, loss: 2.543723, Test Acc: 31.48 %
9 epoch end, loss: 2.409509, Test Acc: 33.14 %
10 epoch end, loss: 2.296386, Test Acc: 36.03 %
11 epoch end, loss: 2.162784, Test Acc: 40.03 %
12 epoch end, loss: 2.042246, Test Acc: 42.43 %
13 epoch end, loss: 1.938158, Test Acc: 39.34 %
14 epoch end, loss: 1.838165, Test Acc: 42.54 %
15 epoch end, loss: 1.757666, Test Acc: 45.11 %
16 epoch end, loss: 1.687990, Test Acc: 47.02 %
17 epoch end, loss: 1.623376, Test Acc: 48.46 %
18 epoch end, loss: 1.566662, Test Acc: 47.61 %
19 epoch end, loss: 1.516931, Test Acc: 50.21 %
20 epoch end, loss: 1.471805, Test Acc: 51.36 %
21 epoch end, loss: 1.425209, Test Acc: 53.77 %
22

171 epoch end, loss: 0.006993, Test Acc: 71.73 %
172 epoch end, loss: 0.006671, Test Acc: 71.52 %
173 epoch end, loss: 0.007108, Test Acc: 71.62 %
174 epoch end, loss: 0.006216, Test Acc: 71.67 %
175 epoch end, loss: 0.006383, Test Acc: 71.61 %
176 epoch end, loss: 0.006236, Test Acc: 71.82 %
177 epoch end, loss: 0.005974, Test Acc: 71.76 %
178 epoch end, loss: 0.006133, Test Acc: 71.78 %
179 epoch end, loss: 0.005700, Test Acc: 71.67 %
180 epoch end, loss: 0.006415, Test Acc: 71.63 %

Training is finished!


In [9]:
correct = 0
total = 0
net.TestMode()
for data in testloader:
    images, labels = data
    outputs = net(Variable(images.cuda()))
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels.cuda()).sum()

print('Accuracy of the network on the 10000 test images: %4.2f %%' % (100 * correct / total))

Accuracy of the network on the 10000 test images: 71.63 %


In [10]:
torch.save(net.GetStateDict(), pretrained_model)